In [3]:
from exjobb_data import *


Using TensorFlow backend.


exjobb data prepare loading...   
data loaded ....


In [4]:
item2vec_data = pd.read_pickle('/home/rocket/dev/jupyter/datasets/item2vec_dataset.pkl')

In [5]:
train_contexts = item2vec_data[0]

In [6]:
test_contexts = item2vec_data[1]

In [7]:
train_vehicles =  item2vec_data[2]

In [8]:
test_vehicles = item2vec_data[3]

In [9]:
train_vehicles[0]

['__PNO12__246_68_R5_C',
 '__OPT__201924668R5C001050',
 '__UPH__201924668R5CRB0R00',
 '__COL__201924668R5C71700']

In [10]:
#import exjobb_data
#from importlib import reload
#reload(exjobb_data)

In [11]:
solver = SolverData()

In [12]:
ct,eng,sv,gb = solver.parse_pno12_str(train_vehicles[0][0])

In [13]:
train_vehicles_resolved = [solver.resolve_solved_items(x) for x in tqdm_notebook(train_vehicles)]

In [14]:
test_vehicles_resolved = [solver.resolve_solved_items(x) for x in tqdm_notebook(test_vehicles)]

In [15]:
with open('CarBinarizer', 'rb') as f:
    cb = pickle.load(f)
with open('VolvoStats', 'rb') as f:
    stats = pickle.load(f)

In [16]:
types = ['CAR_TYPE','ENGINE','SALES_VERSION','GEARBOX','COL','UPH','OPT']
types_df = ['TYPECODE','ENGINECODE','SALESVERSIONCODE','GEARBOXCODE','COLOUR','UPHOLSTERY','OPT_CODES']

In [17]:
def get_data(cars,onehot=True):
    
    result = [[],[],[],[],[],[],[]]
    for car in tqdm_notebook(cars):
        for t_idx in range(len(types)):
            if onehot:
                result[t_idx] += [cb.types_binarizer[types[t_idx]].transform([car[t_idx]])]
            else:
                result[t_idx] += [car[t_idx]]
                
    return result
        

In [18]:
train_vehicles_oh = get_data(train_vehicles_resolved)


In [19]:
train_vehicles_mapped = get_data(train_vehicles_resolved,False)
test_vehicles_mapped = get_data(test_vehicles_resolved,False)

In [20]:
test_vehicles_oh = get_data(test_vehicles_resolved)

In [21]:
[f'{x}_onehot' for x in types_df]

['TYPECODE_onehot',
 'ENGINECODE_onehot',
 'SALESVERSIONCODE_onehot',
 'GEARBOXCODE_onehot',
 'COLOUR_onehot',
 'UPHOLSTERY_onehot',
 'OPT_CODES_onehot']

In [22]:
train_vehicles_df = pd.DataFrame([pd.Series(x) for x in train_vehicles_oh]).T
train_vehicles_mapped_df = pd.DataFrame([pd.Series(x) for x in train_vehicles_mapped]).T
train_vehicles_df.columns = [f'{x}_onehot' for x in types_df]
train_vehicles_mapped_df.columns = [f'{x}_translated' for x in types_df]

In [23]:
test_vehicles_df = pd.DataFrame([pd.Series(x) for x in test_vehicles_oh]).T
test_vehicles_mapped_df = pd.DataFrame([pd.Series(x) for x in test_vehicles_mapped]).T

test_vehicles_df.columns = [f'{x}_onehot' for x in types_df]
test_vehicles_mapped_df.columns = [f'{x}_translated' for x in types_df]

In [24]:
train_context = pd.DataFrame(train_contexts,columns=user_columns)

In [25]:
test_context = pd.DataFrame(test_contexts,columns=user_columns)

In [26]:
train = train_context.merge(train_vehicles_df, left_index=True, right_index=True)
train = train.merge(train_vehicles_mapped_df, left_index=True, right_index=True)

In [27]:
test_vehicles_df.columns

Index(['TYPECODE_onehot', 'ENGINECODE_onehot', 'SALESVERSIONCODE_onehot',
       'GEARBOXCODE_onehot', 'COLOUR_onehot', 'UPHOLSTERY_onehot',
       'OPT_CODES_onehot'],
      dtype='object')

In [28]:
test = test_context.merge(test_vehicles_df, left_index=True, right_index=True)
test = test.merge(test_vehicles_mapped_df, left_index=True, right_index=True)

In [29]:
train.to_pickle('train_2.pickle')

In [30]:
test.to_pickle('test_2.pickle')

In [31]:
train.shape

(36327, 68)

In [32]:
def remap(series):
    return np.array(list(map(lambda x: x[0],series.values)))
def create_train_test_split(use_data,test_size=0.2):
    X_train,X_test, y_typecode_train,y_typecode_test,y_sv_train,y_sv_test,y_eng_train,y_eng_test,y_gb_train,y_gb_test,\
        y_uph_train,y_uph_test,y_col_train,y_col_test,y_opt_train,y_opt_test= train_test_split(use_data,\
        remap(use_data['TYPECODE_onehot']),\
        remap(use_data['SALESVERSIONCODE_onehot']),\
        remap(use_data['ENGINECODE_onehot']),\
        remap(use_data['GEARBOXCODE_onehot']),\
        remap(use_data['UPHOLSTERY_onehot']),\
        remap(use_data['COLOUR_onehot']),
        remap(use_data['OPT_CODES_onehot']),test_size=test_size)
    return X_train,X_test, y_typecode_train,y_typecode_test,y_sv_train,y_sv_test,y_eng_train,y_eng_test,y_gb_train,y_gb_test,y_uph_train,y_uph_test,y_col_train,y_col_test,y_opt_train,y_opt_test

X_train,X_dev, y_typecode_train,y_typecode_dev,y_sv_train,y_sv_dev,y_eng_train,y_eng_dev,y_gb_train,y_gb_dev,y_uph_dev,y_uph_dev,y_col_train,y_col_dev,y_opt_train,y_opt_dev= create_train_test_split(train,test_size=0.2)

X_train['TYPECODE_translated'].value_counts()
print(X_train.shape)
X_train.to_pickle('train_2.pickle')
X_dev.to_pickle('dev_2.pickle')
#X_train, y_train = merged[user_columns], mlb.transform(df_train['TYPED_CAR'])

(29061, 68)


In [33]:
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler()
for key in types_df:
    if 'OPT' in key:
        continue
    X_res, y_res = ros.fit_resample(X_train, X_train[f'{key}_translated'])
    resampled = pd.DataFrame(X_res,columns=X_train.columns)
    print(key,  resampled[f'{key}_translated'].value_counts())
    print('file',f'{key}_2_train.pickle')
    resampled.to_pickle(f'{key}_2_train.pickle')


TYPECODE 235    9934
256    9934
525    9934
224    9934
225    9934
227    9934
536    9934
526    9934
234    9934
236    9934
246    9934
Name: TYPECODE_translated, dtype: int64
file TYPECODE_2_train.pickle
ENGINECODE 25    9568
35    9568
68    9568
13    9568
72    9568
32    9568
BB    9568
AR    9568
A6    9568
AC    9568
A8    9568
AK    9568
BM    9568
A3    9568
14    9568
Name: ENGINECODE_translated, dtype: int64
file ENGINECODE_2_train.pickle
SALESVERSIONCODE 13    12417
R5    12417
23    12417
N5    12417
12    12417
RI    12417
FS    12417
22    12417
11    12417
R2    12417
R7    12417
Name: SALESVERSIONCODE_translated, dtype: int64
file SALESVERSIONCODE_2_train.pickle
GEARBOXCODE 4    15497
D    15497
F    15497
C    15497
1    15497
0    15497
Name: GEARBOXCODE_translated, dtype: int64
file GEARBOXCODE_2_train.pickle
COLOUR 72400    3503
47700    3503
70700    3503
01900    3503
46700    3503
71200    3503
72700    3503
62100    3503
72600    3503
71400    3503
72000  

In [34]:
test.to_pickle('test_2.pickle')

In [35]:
train.columns

Index(['antal_inpendlare', 'antal_utpendlare', 'förvärvs-arbetande',
       'ej_förvärvs-arbetande', 'äganderätt/småhus', 'bostadsrätt',
       'hyresrätt', 'övriga_inkl._uppgift_saknas', 'förgymnasial', 'gymnasial',
       'eftergymnasial_mindre_än_3_år',
       'eftergymnasial_3_år_eller_längre_inkl._forskarutbildning', '0-6_år',
       '7-15_år', '16-19_år', '20-24_år', '25-44_år', '45-64_år', '65-w_år',
       'låg_inkomst', 'medellåg_inkomst', 'medelhög_inkomst', 'hög_inkomst',
       'medianinkomst', 'sammanboende_med_barn', 'sammanboende_utan_barn',
       'ensamstående_med_barn', 'ensamstående_utan_barn', 'övriga_hushåll',
       'låg_köpkraft', 'medellåg_köpkraft', 'medelhög_köpkraft',
       'hög_köpkraft', 'median_köpkraft',
       'jordbruk,_skogsbruk,_jakt_och_fiske', 'tillverkning_och_utvinning',
       'energi_och_miljöverksamhet', 'byggverksamhet', 'handel',
       'transport_och_magasinering', 'hotell-_och_restaurangverksamhet',
       'information_och_kommunikation', 

In [36]:
test.columns

Index(['antal_inpendlare', 'antal_utpendlare', 'förvärvs-arbetande',
       'ej_förvärvs-arbetande', 'äganderätt/småhus', 'bostadsrätt',
       'hyresrätt', 'övriga_inkl._uppgift_saknas', 'förgymnasial', 'gymnasial',
       'eftergymnasial_mindre_än_3_år',
       'eftergymnasial_3_år_eller_längre_inkl._forskarutbildning', '0-6_år',
       '7-15_år', '16-19_år', '20-24_år', '25-44_år', '45-64_år', '65-w_år',
       'låg_inkomst', 'medellåg_inkomst', 'medelhög_inkomst', 'hög_inkomst',
       'medianinkomst', 'sammanboende_med_barn', 'sammanboende_utan_barn',
       'ensamstående_med_barn', 'ensamstående_utan_barn', 'övriga_hushåll',
       'låg_köpkraft', 'medellåg_köpkraft', 'medelhög_köpkraft',
       'hög_köpkraft', 'median_köpkraft',
       'jordbruk,_skogsbruk,_jakt_och_fiske', 'tillverkning_och_utvinning',
       'energi_och_miljöverksamhet', 'byggverksamhet', 'handel',
       'transport_och_magasinering', 'hotell-_och_restaurangverksamhet',
       'information_och_kommunikation', 